In [486]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
data_dir = '/Users/shivyucel/Documents/projects/opioid_accessibility/opioid_accessibility/data/toronto_data/'

In [487]:
DA_shp = gpd.read_file(data_dir + 'toronto_shapes/DA_geometries/DA_shp.shp')

In [488]:
toronto_proper = DA_shp[DA_shp['CD_UID'] == '3520']

In [513]:
da_users = pd.read_csv(data_dir + 'da_users.csv')
travel_times = gpd.read_file(data_dir + 'results/travel_time_diff/travel_time_diff_2.shp')
msdi = pd.read_excel(data_dir + 'deprivation_data/MSDI.xlsx', sheet_name='DATA')

/Users/shivyucel/opt/anaconda3/envs/geo_env/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [514]:
clean_times = gpd.read_file(data_dir + 'clean_files/clean_times/clean_times.shp')

In [515]:
clean_times.crs

<Derived Projected CRS: EPSG:3347>
Name: NAD83 / Statistics Canada Lambert
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Canada - onshore and offshore - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon.
- bounds: (-141.01, 38.21, -40.73, 86.46)
Coordinate Operation:
- name: Statistics Canada Lambert
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [516]:
toronto_proper['GeoUID'] = toronto_proper['GeoUID'].astype(int)

/Users/shivyucel/opt/anaconda3/envs/geo_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [517]:
msdi = msdi.merge(toronto_proper, left_on='DA', right_on='GeoUID')

In [518]:
msdi['mat_quant'] = pd.qcut(msdi['SCOREMAT'], 5, labels=False)
msdi['soc_quant'] = pd.qcut(msdi['SCORESOC'], 5, labels=False) 


msdi['mat_quant'] = msdi['mat_quant'] + 1
msdi['soc_quant'] = msdi['soc_quant'] + 1

In [519]:
msdi = msdi[['DA', 'mat_quant', 'soc_quant', 'DAPOP2016']]

In [520]:
msdi['mat_quant'].value_counts()

5.0    716
1.0    716
4.0    715
3.0    715
2.0    715
Name: mat_quant, dtype: int64

In [521]:
full_table = travel_times.merge(msdi, left_on='GeoUID', right_on='DA')

In [522]:
full_table = full_table.merge(da_users, left_on='GeoUID', right_on='DAUID', how='left')

In [523]:
full_table['da_opioid_users'].replace(np. nan,0, inplace=True) 

In [524]:
clean_times = clean_times.merge(da_users, on='DAUID', how='left')

In [525]:
clean_times['da_opioid_users'].replace(np. nan,0, inplace=True) 

In [526]:
clean_times = clean_times.merge(msdi, left_on='DAUID', right_on='DA')

In [527]:
clean_times.rename(columns={'da_opioid_users':'opioid_pop'}, inplace=True)

In [528]:
clean_times.drop('Unnamed: 0', axis=1, inplace=True)

In [529]:
clean_times.to_file(data_dir + 'clean_files/clean_times_v2/clean_times_v2.shp')

In [532]:
clean_times

,wed_0800,sat_0900,wed_1100,wed_1500,DAUID,reg_name,area_km,pop,avg_inc,pop_nocar,low_inc,geometry,opioid_pop,DA,mat_quant,soc_quant,DAPOP2016
0,0.0,0.0,-3.0,-1.0,35200002,Toronto,0.05037,356,79431.0,55.071330,25.0,"POLYGON ((7235605.171 950635.097, 7235659.626 ...",1.726067,35200002,4.0,2.0,356.0
1,1.0,0.0,-3.0,-1.0,35200003,Toronto,0.04675,335,97695.0,0.000000,35.0,"POLYGON ((7235794.929 950295.603, 7235846.557 ...",1.624248,35200003,3.0,1.0,335.0
2,0.0,0.0,-3.0,-2.0,35200004,Toronto,0.04878,384,66349.0,0.000000,40.0,"POLYGON ((7235509.514 950062.337, 7235507.757 ...",1.861825,35200004,5.0,1.0,384.0
3,-1.0,0.0,-3.0,-1.0,35200005,Toronto,0.04429,408,97356.0,0.000000,25.0,"POLYGON ((7235628.734 949921.829, 7235606.546 ...",1.978189,35200005,5.0,1.0,408.0
4,-1.0,0.0,-3.0,-1.0,35200006,Toronto,0.05717,527,80188.0,0.000000,40.0,"POLYGON ((7235705.871 949866.543, 7235688.483 ...",2.555161,35200006,4.0,1.0,527.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697,0.0,0.0,0.0,0.0,35204902,Toronto,0.01994,498,55037.0,321.265496,165.0,"POLYGON ((7224597.117 929588.000, 7224434.543 ...",102.996009,35204902,1.0,4.0,498.0
3698,2.0,3.0,2.0,3.0,35204903,Toronto,0.63493,3495,48495.0,494.285900,1170.0,"POLYGON ((7222961.389 941366.423, 7222950.120 ...",30.687147,35204903,2.0,4.0,3495.0
3699,2.0,2.0,1.0,2.0,35204904,Toronto,0.08969,626,51243.0,292.351880,175.0,"POLYGON ((7222913.800 941347.529, 7222901.400 ...",5.496467,35204904,3.0,4.0,626.0
3700,1.0,1.0,1.0,1.0,35204905,Toronto,1.08844,1774,50783.0,307.684076,525.0,"POLYGON ((7225992.023 942215.857, 7225991.686 ...",97.502994,35204905,2.0,5.0,1774.0


In [530]:
len(full_table['fromId'].unique())

3699

In [437]:
df_0800 = full_table[full_table['time'] == '0800']
df_1100 = full_table[full_table['time'] == '1100']
df_1500 = full_table[full_table['time'] == '1500']
df_0900_sat = full_table[full_table['time'] == '0900_sat']

all_dfs = {'0800': df_0800, '1100': df_1100, '1500': df_1500, '0900_sat': df_0900_sat}

In [438]:
len(full_table['fromId'].unique())

3699

In [471]:
results_mat = {}
for key in all_dfs.keys():
    df = all_dfs[key]
    
    thres_20 = df[(df['tt_2019'] < 20) & (df['tt_2020'] >= 20)]
    thres_30 = df[(df['tt_2019'] < 30) & (df['tt_2020'] >= 30)]
    increase_10 = df[(df['diff'] >= 10)]
    


    count_da_thres_20 = thres_20.groupby('mat_quant')['fromId'].count().reset_index()
    count_pop_thres_20 = thres_20.groupby('mat_quant')[['DAPOP2016', 'da_opioid_users']].sum().reset_index()

    results_mat[key + '_thres_20'] = count_da_thres_20.merge(count_pop_thres_20, on='mat_quant')
    
    

    
    count_da_thres_30 = thres_30.groupby('mat_quant')['fromId'].count().reset_index()
    count_pop_thres_30 = thres_30.groupby('mat_quant')[['DAPOP2016', 'da_opioid_users']].sum().reset_index()
    
    results_mat[key + '_thres_30'] = count_da_thres_30.merge(count_pop_thres_30, on='mat_quant')
    
    
    

    count_da_increase_10 = increase_10.groupby('mat_quant')['fromId'].count().reset_index()
    count_pop_increase_10 = increase_10.groupby('mat_quant')[['DAPOP2016', 'da_opioid_users']].sum().reset_index()
    
    results_mat[key + '_increase_10'] = count_da_increase_10.merge(count_pop_increase_10, on='mat_quant')

In [472]:
results_mat.keys()

dict_keys(['0800_thres_20', '0800_thres_30', '0800_increase_10', '1100_thres_20', '1100_thres_30', '1100_increase_10', '1500_thres_20', '1500_thres_30', '1500_increase_10', '0900_sat_thres_20', '0900_sat_thres_30', '0900_sat_increase_10'])

In [473]:
result_df = pd.DataFrame()

for key in results_mat.keys():
    results_mat[key]['time'] = key
    result_df = result_df.append(results_mat[key])
    

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_48246/1281457017.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(results_mat[key])
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_48246/1281457017.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(results_mat[key])
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_48246/1281457017.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(results_mat[key])
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_48246/1281457017.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

In [474]:
result_df['time'].unique()

array(['0800_thres_20', '0800_thres_30', '0800_increase_10',
       '1100_thres_20', '1100_thres_30', '1500_thres_20', '1500_thres_30',
       '1500_increase_10', '0900_sat_thres_20', '0900_sat_thres_30'],
      dtype=object)

In [477]:
result_df[result_df['time'] == '1100_increase_10']

,mat_quant,fromId,DAPOP2016,da_opioid_users,time


# soc_quant

In [478]:
results_soc = {}
for key in all_dfs.keys():
    df = all_dfs[key]
    
    thres_20 = df[(df['tt_2019'] < 20) & (df['tt_2020'] >= 20)]
    thres_30 = df[(df['tt_2019'] < 30) & (df['tt_2020'] >= 30)]
    increase_10 = df[(df['diff'] >= 10)]
    


    count_da_thres_20 = thres_20.groupby('soc_quant')['fromId'].count().reset_index()
    count_pop_thres_20 = thres_20.groupby('soc_quant')[['DAPOP2016', 'da_opioid_users']].sum().reset_index()

    results_soc[key + '_thres_20'] = count_da_thres_20.merge(count_pop_thres_20, on='soc_quant')
    
    

    
    count_da_thres_30 = thres_30.groupby('soc_quant')['fromId'].count().reset_index()
    count_pop_thres_30 = thres_30.groupby('soc_quant')[['DAPOP2016', 'da_opioid_users']].sum().reset_index()
    
    results_soc[key + '_thres_30'] = count_da_thres_30.merge(count_pop_thres_30, on='soc_quant')
    
    
    

    count_da_increase_10 = increase_10.groupby('soc_quant')['fromId'].count().reset_index()
    count_pop_increase_10 = increase_10.groupby('soc_quant')[['DAPOP2016', 'da_opioid_users']].sum().reset_index()
    
    results_soc[key + '_increase_10'] = count_da_increase_10.merge(count_pop_increase_10, on='soc_quant')

In [479]:
result_df_soc = pd.DataFrame()

for key in results_soc.keys():
    results_soc[key]['time'] = key
    result_df_soc = result_df_soc.append(results_soc[key])
    

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_48246/2028971951.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_soc = result_df_soc.append(results_soc[key])
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_48246/2028971951.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_soc = result_df_soc.append(results_soc[key])
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_48246/2028971951.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_soc = result_df_soc.append(results_soc[key])
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_48246/2028971951.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

In [480]:
result_df_soc['time'].unique()

array(['0800_thres_20', '0800_thres_30', '0800_increase_10',
       '1100_thres_20', '1100_thres_30', '1500_thres_20', '1500_thres_30',
       '1500_increase_10', '0900_sat_thres_20', '0900_sat_thres_30'],
      dtype=object)

In [485]:
result_df_soc[result_df_soc['time'] == '1500_increase_10']

,soc_quant,fromId,DAPOP2016,da_opioid_users,time
0,1.0,1,1240.0,6.012142,1500_increase_10
1,3.0,1,451.0,14.484511,1500_increase_10
